In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import os
import matplotlib.pyplot as plt
import IPython.display as ipd
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

import efficientnet.tfkeras as efn #Convolutional Neural Network architecture
from efficientnet.tfkeras import center_crop_and_resize, preprocess_input
from sklearn.utils import class_weight

In [34]:
img_size = (4096, 1024)
batch_size = 16

In [35]:
storage_dir = "./data/train_img/"
existing_birds = []

for root, dirs, files in os.walk(storage_dir):
    if root == storage_dir:
        existing_birds = dirs

In [36]:
# References:
# https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)

train_batches = train_datagen.flow_from_directory(
    storage_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    shuffle=True,
    classes=existing_birds
)

valid_batches = train_datagen.flow_from_directory(
    storage_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    shuffle=False,
    classes=existing_birds
)

Found 17212 images belonging to 264 classes.
Found 4240 images belonging to 264 classes.


In [37]:
# References:
# https://keras.io/api/applications/efficientnet/
# https://github.com/qubvel/efficientnet
net = efn.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(4096, 1024, 1)
)

x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(existing_birds), activation='softmax')(x)

# https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/
net_final = Model(inputs=net.input, outputs=output_layer)     
net_final.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [38]:
print(net_final.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 190, 190, 48) 1296        input_4[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalizationV1)  (None, 190, 190, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 190, 190, 48) 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

In [39]:
# Estimate class weights for unbalanced dataset
class_weights = class_weight.compute_class_weight(
    'balanced',
    np.unique(train_batches.classes), 
    train_batches.classes
)

class_weights = {i : class_weights[i] for i in range(len(class_weights))}

In [40]:
# Define callbacks
model_check = ModelCheckpoint(
    'models/efficientnet_checkpoint.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto', 
    period=1
)

reduce_LR = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=5, 
    min_lr=3e-4
)

In [ ]:
# Train the model
net_final.fit_generator(
    train_batches,
    validation_data = valid_batches,
    epochs = 40,
    steps_per_epoch = 1596,
    class_weight=class_weights,
    callbacks=[model_check, reduce_LR]
)